# 다양한 필터링을 활용한 영화 추천 시스템

TMDB 데이터 셋의 EDA을 통해 데이터의 파악을 마쳤다. 그렇다면 분석이 완료된 데이터들을 활용해 사용자에게 알맞은 영화를 추천해 줄 수 있는 시스템들을 구상해보자.  
  
이번 연구에서 구상한 추천 시스템은 총 5개이다. 단순히 인기가 많은 영화를 추천하는 간단한 시스템부터, 개별 영화와 사용자의 성향을 파악하여 추천해주는 보다 복잡한 시스템까지 구현해보았다.  
  
시스템들을 구성 및 구현하는 일련의 과정을 나열함으로써 시스템마다 발전시킨 부분과 그 부분들로 인해 나타난 결과(추천 목록)의 차이를 바로 볼 수 있게 하였다. 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise import KNNBasic
from surprise.model_selection import cross_validate as cross
from surprise.model_selection import train_test_split

import warnings; warnings.simplefilter('ignore')

## 1. Simple 영화 추천 시스템

In [2]:
md = pd. read_csv('./movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

TMDB의 Rating 데이터 셋을 통해 *가중치* 공식을 사용하여 **상위 영화 차트**를 생성할 것이다. 가중치 공식은 수학적으로 다음과 같이 표현된다.

Weighted Rating(WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

* *v*는 영화에 대한 vote 수.
* *m*은 차트에 사용하기 위해 필요한 최소 vote 수.
* *R*는 영화의 평균 rating.
* *C*는 전체 보고서에 걸친 평균 vote 수.

다음 단계는 차트에 나열되는 데 필요한 최소 vote인 *m*에 대한 적절한 값을 결정하는 것이다. 여기서는 **상위 5%** 를 기준으로 데이터의 삽입 여부를 결정할 것이다. 다시 말해, 차트에서 영화가 등록되려면, 리스트에 있는 영화들 중 하위 95% 이상의 표(상위 5% 만큼의 표)를 얻어야 한다는 것이다.

이제 전체 Top 250 차트를 만들고 특정 장르를 위한 차트를 만드는 기능을 정의할 것이다.

In [4]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [5]:
m = vote_counts.quantile(0.95)
m

434.0

In [6]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [7]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

따라서, 차트에 오르려면, 영화는 TMDB에서 최소한 434의 vote를 얻어야 한다. 또한 우리는 TMDB에서 영화의 평균 평점이 10점 만점에 5.244인 것과 2274편의 영화가 우리 차트에 오를 수 있다는 것을 확인 했다.

In [8]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [9]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [10]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

#### 실행 결과-상위 영화 차트

In [11]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


가장 인기가 많은 영화들로 구성된 상위 영화 차트가 완성되었다. 

이제 특정 장르를 위한 차트를 만드는 기능을 구성해보자. 이를 위해 우리는 상위 5% 아닌 상위 15%로 default condition을 완화시킬 것이다.

In [12]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [13]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

그렇다면 Romance 장르(인기가 많은 장르임에도 불구하고 상위 포지션에 위치하지 못했으므로)를 예시로 실행해보자.

#### 실행 결과-장르별 상위 영화 차트(Romance)

In [14]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


첫번째 시스템은 Vote의 개수와 Rating만을 사용했기 때문에 "추천" 시스템이라고 하기엔 부족한 면이 많다. 그렇다면 본격적으로 영화 "추천" 시스템을 구현해보자.

## 2. Pearson correlation을 활용한 추천 시스템

이번 연구에서 활용하고 있는 데이터 셋에는 약 2600만개의 사용자들이 평가한 영화들의 평점들이 존재한다. 이러한 평점들을 집중적으로 활용한 추천시스템을 구현해 보고자 한다.  

여기서는 피어슨 상관 계수를 사용할 예정이다. 피어슨 상관계수를 적용할 수 있는 이유는 아래와 같다.  

만약 두 사람이 보는 영화들이 비슷하다면 영화 취향 역시 비슷할 것이다. 그렇다면 그 두 사람 중 한 사람이 평가한 특정 영화에 대해 좋은 평가를 내린다면, 다른 사람 역시 좋은 평가를 내릴 수 있다고 유추해볼 수 있다. 이러한 방법을 통해 시스템을 구상해보고자 한다.  

In [15]:
meta = pd.read_csv('./movies_metadata.csv')

meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta = meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language'] == 'en']
meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [17]:
ratings = pd.read_csv('./ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [18]:
#Dataset 가공
meta.movieId = pd.to_numeric(meta.movieId, errors = 'coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors = 'coerce')

In [19]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres)

meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[Animation, Comedy, Family]"
1,8844,Jumanji,en,"[Adventure, Fantasy, Family]"
2,15602,Grumpier Old Men,en,"[Romance, Comedy]"
3,31357,Waiting to Exhale,en,"[Comedy, Drama, Romance]"
4,11862,Father of the Bride Part II,en,[Comedy]


In [20]:
#사용자 평가(rating)과 영화 정보(meta)를 합침 -> 영화에 대한 사용자 rating을 추려줌
data = pd.merge(ratings, meta, on='movieId', how='inner')

data.head()

,userId,movieId,rating,original_title,original_language,genres
0,1,858,5.0,Sleepless in Seattle,en,"[Comedy, Drama, Romance]"
1,3,858,4.0,Sleepless in Seattle,en,"[Comedy, Drama, Romance]"
2,5,858,5.0,Sleepless in Seattle,en,"[Comedy, Drama, Romance]"
3,12,858,4.0,Sleepless in Seattle,en,"[Comedy, Drama, Romance]"
4,20,858,4.5,Sleepless in Seattle,en,"[Comedy, Drama, Romance]"


In [21]:
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

matrix.head(20)

original_title,!Women Art Revolution,$5 a Day,'Gator Bait,'R Xmas,'Twas the Night Before Christmas,(A)Sexual,...And the Pursuit of Happiness,10 Items or Less,10 Things I Hate About You,"10,000 BC",...,Мой сводный брат Франкенштейн,Седьмой спутник,"Цирк сгорел, и клоуны разбежались",به امید دیدار,مارمولک,რამინი,常在我心,軍旗はためく下に,黑太陽731,태풍
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#피어슨 상관계수
GENRE_WEIGHT = 0.1

def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]

    result = []
    for title in matrix.columns:
        if title == input_movie:#똑같은 movie를 추천 받는다는 거니까 뛰어넘음
            continue

        # rating comparison
        cor = pearsonR(matrix[input_movie], matrix[title])
        
        # genre comparison
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]

            same_count = np.sum(np.isin(input_genres, temp_genres))#같은 장르가 많을수록 가중치 부여(np.is1d():
            cor += (GENRE_WEIGHT * same_count)#가중치 1 부여   //배열을 비교하여 똑같은 요소가 있으면 True를 반환함.)
        
        if np.isnan(cor):
            continue
        else:
            result.append((title, '{:.2f}'.format(cor), temp_genres))
            
    result.sort(key=lambda r: r[1], reverse=True)#rating이 높은 순에서 낮은순으로
        #                                                   내림차순 정렬

    return result[:n]#입력한 개수만큼 반환

#### 실행 결과-Pearson Correlation을 활용한 시스템

In [23]:
recommend_result = recommend('The Dark Knight', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

,Title,Correlation,Genre
0,Payback,0.44,"[Drama, Action, Thriller, Crime]"
1,48 Hrs.,0.43,"[Thriller, Action, Comedy, Crime, Drama]"
2,Certain Prey,0.43,"[Crime, Drama, Action, Thriller, Mystery]"
3,Hitman,0.43,"[Action, Crime, Drama, Thriller]"
4,The Contract,0.43,"[Drama, Action, Thriller, Crime]"
5,The Racket,0.43,"[Drama, Action, Thriller, Crime]"
6,The Way of the Gun,0.43,"[Action, Crime, Drama, Thriller]"
7,Armored,0.42,"[Action, Crime, Drama, Thriller]"
8,Bullitt,0.42,"[Action, Crime, Drama, Thriller]"
9,Dark Blue,0.42,"[Action, Crime, Drama, Thriller]"


이번 시스템은 사용자들의 Rating과 장르를 기반으로 만들어졌기 때문에, 1번 시스템에 비해 정교하게 영화를 추천을 할 수 있다. 하지만 이 시스템 역시 영화를 추천하는 척도로 Rating과 장르만을 사용했기 때문에 어느정도 부족한 면이 있다. 영화의 다른 요소들을 활용하여 또 다른 시스템을 구상해보자.

## 3. 콘텐츠 기반 추천 시스템

첫번째로 만든 간단한 영화 추천 시스템은 몇 가지 심각한 문제점이 있다. 우선, 사용자의 **개인적 취향**에 상관없이 모든 사람에게 동일한 영화들을 추천한다는 것이다. 만약 로맨틱 영화를 좋아하고 액션을 싫어하는 사람이 이 추천 시스템의 상위 15위 차트를 본다면, 그는 아마 대부분의 영화를 좋아하지 않을 것이다. 한 걸음 더 나아가 장르별로 우리 차트를 살펴본다 하더라도, 영화의 많은 요소들을 포함하지 않기 때문에, 여전히 만족할 만한 추천을 받지 못할 가능성이 크다.

따라서 보다 추천을 좀 더 개인화(personalize)하기 위해 특정 요소들을 바탕으로 영화 간 유사성을 계산하고 사용자가 좋아했던 특정 영화와 가장 유사한 영화를 제안하는 추천 시스템을 만들 생각이다. 영화 메타데이터(metadata)를 사용할 것이기 때문에, 이를 **콘텐츠 기반 추천 시스템**이라고 지칭 하겠다.

다음을 기반으로 두 개의 콘텐츠 기반 추천 시스템을 구축한다.

*영화 Overviews 및 tagline(슬로건 같은 문구)  
*영화 출연진, 제작진, 키워드 및 장르

In [24]:
links = pd.read_csv('./links.csv')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [25]:
md = md.drop([19730, 29503, 35587])

In [26]:
#EDA를 진행한 부분에서 왜 이런 index들이 나왔는지 확인 가능
md['id'] = md['id'].astype('int')

In [27]:
smd = md[md['id'].isin(links)]
smd.shape

(42845, 25)

### 3-1. 영화 설명 기반 추천 시스템

우선 영화 설명(Movie Overview)과 Tagline으로 추천 시스템을 만들어 보자. 현재 정량적 측정 기준을 가지고 있지 않기 때문에, 정성적으로 측정이 이루어져야 할 것이다.

In [28]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [29]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [30]:
tfidf_matrix.shape

(42845, 1049589)

#### Cosine 유사도
Cosine 유사도를 이용하여 두 영화의 유사성을 나타내는 numerical quantity를 계산할 것이다. 이는 수학적으로 다음과 같이 정의된다.  

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $  

TF-IDF Vectorizer를 사용했기 때문에 Dot Product를 계산하면 코사인 유사도 점수가 나온다. 따라서 우리는 훨씬 속도가 빠른 sklearn의 linear_kernel을 사용할 것이다. 

In [31]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [32]:
cosine_sim[0]

array([1.        , 0.00515426, 0.        , ..., 0.        , 0.00238517,
       0.        ])

이제 데이터 셋의 모든 영화에 대해 쌍방향 Cosine 유사도 매트릭스를 가지고 있다. 다음 단계는 Cosine 유사도 점수를 바탕으로 가장 유사한 영화 30편을 반환하는 기능을 구현하는 것이다.

In [33]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [34]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

이제 몇 편의 영화에 대한 상위 추천을 받아보고 추천이 얼마나 잘 되는지 확인하자.
#### 실행 결과-콘텐츠 기반 추천 시스템1 

In [35]:
get_recommendations('The Godfather').head(10)

1173       The Godfather: Part II
29481            Honor Thy Father
20046                  The Family
21107                  Blood Ties
35488    A Mother Should Be Loved
17385             The Outside Man
10605                    Election
11050            Household Saints
4313                         Made
5422           Johnny Dangerously
Name: title, dtype: object

In [36]:
get_recommendations('Star Wars').head(10)

27994                   The Star Wars Holiday Special
1149                          The Empire Strikes Back
1162                               Return of the Jedi
24160                    Star Wars: The Force Awakens
20396                              The Galaxy Invader
20961                              Threads of Destiny
31650      Samson and the Seven Miracles of the World
23850                           Princess and the Pony
9898     Star Wars: Episode III - Revenge of the Sith
11543                                 Shrek the Third
Name: title, dtype: object

스타워즈를 확인해보면, 현재의 시스템이 그것을 스타워즈 시리즈 영화라고 식별하고 그 후에 다른 스타워즈 영화들을 추천작으로 추천하는 것을 확인할 수 있다. 불행히도 이 시스템이 현재 할 수 있는 일은 이정도 뿐이다. 이것은 영화의 평점과 인기를 좌우하는 출연진, 제작진, 감독, 장르와 같은 매우 중요한 특징들을 고려하지 않기 때문이고, 이는 곧 대부분의 사람들에게 그다지 유용하지 않을 수 있다는 것을 시사한다.

따라서 Overview와 Tagline보다 훨씬 더 유용한 메타데이터를 사용(or 추가)해야 한다. 다음 섹션에서는 장르, 키워드, 출연진, 제작진 등을 고려한 보다 정교한 추천 시스템을 구축할 것이다.

### 3-2. Metadata 기반 추천 시스템  

3-1에서 나아가, 메타데이터 기반 콘텐츠 추천 시스템을 구축하려면 현재 데이터 셋을 제작진 및 키워드 데이터셋과 병합해야 한다.

In [37]:
credits = pd.read_csv('./credits.csv')
keywords = pd.read_csv('./keywords.csv')

In [38]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [39]:
md.shape

(45463, 25)

In [40]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [41]:
smd = md[md['id'].isin(links)]
smd.shape

(43997, 28)

이제 출연진, 제작진, 장르, 크레딧을 모두 하나의 데이터 프레임으로 가지고 있다. 다음의 방법들을 통해 조금 더 나아가보자.  

1. 제작진(Crew): 다른 제작진들은 영화의 느낌에 별로 기여하지 않기 때문에, 제작진 중에서 감독만을 추출한다.
2. 출연자(Cast): 출연자는 제작진보다 좀 더 까다롭다. 덜 알려진 배우들과 단역 배우들은 실제로 영화에 대한 사람들의 의견에 영향을 미치지 않는다. 그러므로 우리는 주요 인물과 각각의 배우만을 선정해야 한다. 임의로 크레딧 리스트에 등장하는 상위 3명의 배우를 선정할 것이다.

In [42]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [43]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [44]:
smd['director'] = smd['crew'].apply(get_director)

In [45]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [46]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

내가 계획하고 있는 것은 장르, 감독, 주연배우, 키워드로 구성된 모든 영화의 metadata dump를 만들고, 앞에서와 같이 Count Vectorizer를 사용하여 count matrix를 생성하는 것이다.

이에 따라 나머지 단계들은 우리가 앞서 했던 것과 유사하다. 코사인 유사도를 계산하고 가장 유사한 영화를 반환하는 것이다.

장르 및 Credit 데이터를 준비할 때 따라야 할 단계는 다음과 같다.  

1. 띄어쓰기를 분리하고 소문자로 변환한다. 이렇게 하면 시스템은 Johnny Depp과 다른 Johnny들을 혼동하지 않을 것이다.  
2. 전체 출연진에 비해 가중치를 주기 위해 감독을 2번 언급한다.

In [47]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [48]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x])

#### 키워드  
키워드를 사용하기 전에 소량의 pre-processing을 할 것이다. 첫 번째 단계로 데이터 셋에 나타나는 모든 키워드의 frequency를 계산한다.

In [49]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [50]:
s = s.value_counts()
s[:5]

woman director      2993
independent film    1860
murder              1262
based on novel       819
musical              714
Name: keyword, dtype: int64

여기서 한 번만 발생하는 키워드는 사용하지 않을 것이다.  마지막으로, Dogs와 Dog와 같은 단어들이 똑같이 여겨지도록 모든 단어를 stem으로 변환할 것이다.

In [51]:
s = s[s > 1]

In [52]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [53]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [54]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [55]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [56]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [57]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [58]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

코사인 유사도 점수가 바뀌었기 때문에, 아마 다른 결과를 줄 것으로 기대한다. 이에 따라 앞에서 작성한 get_recommendations 기능을 재사용하여 스타워즈를 다시 한 번 확인해보고, 이번에는 어떤 추천을 받았는지 보자.

#### 실행 결과-콘텐츠 기반 추천 시스템2(제작진, 출연진 추가)

In [59]:
get_recommendations('Star Wars').head(10)

1170                          The Empire Strikes Back
2527        Star Wars: Episode I - The Phantom Menace
1183                               Return of the Jedi
5281     Star Wars: Episode II - Attack of the Clones
9986     Star Wars: Episode III - Revenge of the Sith
22529               Electronic Labyrinth THX 1138 4EB
41712                                Toward the Terra
23533                                        Fireball
38308                                     The Call Up
24585                                      Last Rites
Name: title, dtype: object

나는 이번에 얻은 결과에 훨씬 더 만족한다. 이번 추천은 감독의 다른 영화들을 (감독에게 주어진 높은 무게 때문에) 인식하여 최고의 추천작으로 올린 것으로 보인다.

여러 특징(감독, 배우, 장르)에 따라 다른 가중치를 시험해 보고, soup에 사용할 수 있는 키워드 수를 제한하고, frequency에 따라 장르의 가중치를 부여하고, 같은 언어로 된 영화만 제시하는 등의 방법으로 실험을 이어나갈 수도 있다.

In [60]:
get_recommendations('Pulp Fiction').head(10)

39813    My Best Friend's Birthday
1661                  Jackie Brown
25930            The Hateful Eight
7327             Kill Bill: Vol. 2
6181                         Basic
20863             Reasonable Doubt
6502                      S.W.A.T.
12120                      Cleaner
27076                   The Forger
255                  Kiss of Death
Name: title, dtype: object

#### 3.3 인기도(Popularity)와 평점(Ratings)

구상한 콘텐츠 기반 추천 시스템을 들여다보면, 인기도와 평점에 상관없이 영화를 추천하도록 설계되었다.

따라서 좀 더 나아가 이번 섹션에서는 비판적이고 나쁜 영화들을 제거하는 동시에, 인기 있고 반응이 좋았던 영화를 돌려줄 수 있는 메커니즘을 추가하겠다.

유사도 점수를 바탕으로 나온 상위 25개 영화에 vote count의 백분위를 60%로 계산하겠다. 그런 다음 이것을 $m$ 의 값으로 삼아 단순 추천 시스템 섹션에서와 같이 가중치 공식을 이용하여 각 영화의 Weighted rating을 산출해 볼 것이다.

In [61]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

#### 실행 결과-콘텐츠 기반 추천 시스템3(인기도, 평점 추가)

In [62]:
improved_recommendations('Star Wars')

,title,vote_count,vote_average,year,wr
1170,The Empire Strikes Back,5998,8,1980,7.814099
24374,Star Wars: The Force Awakens,7993,7,2015,6.909610
1183,Return of the Jedi,4763,7,1983,6.853432
9986,Star Wars: Episode III - Revenge of the Sith,4200,7,2005,6.835625
2527,Star Wars: Episode I - The Phantom Menace,4526,6,1999,5.933928
3687,X-Men,4172,6,2000,5.928850
5281,Star Wars: Episode II - Attack of the Clones,4074,6,2002,5.927304
21722,Ra.One,89,5,2011,5.203222
12634,Star Wars: The Clone Wars,434,5,2008,5.122448
17785,Journey 2: The Mysterious Island,1050,5,2012,5.071621


In [ ]:
improved_recommendations('Pulp Fiction')

추가한 메커니즘에 따라 결과값이 바뀐 것을 확인할 수 있다.

## 4. 협업 필터링 추천 시스템

지금까지 제작한 콘텐트 기반 추천 시스템은 몇 가지 심각한 제약이 있다. 특정 영화에 가까운 영화만 제안할 수 있고, 사용자의 취향을 포착하고 장르를 넘나드는 추천을 제공할 수 없다는 것이다.

이전 섹션들의 추천 시스템은 어느 사람이든 특정 영화를 입력하면 똑같은 추천을 받기 때문에, 사용자의 개인적 취향을 포착하지 못한다는 점에서 personal하지 못하다.

따라서 이 섹션에서는 협업 필터링이라는 기술을 사용하여 조금 더 정교한 추천을 구상할 것이다. 사용하고 있는 데이터 셋에 방대한 양의 유저 정보들이 들어있으므로, 이러한 시스템이 구상 가능하다.

Surprise 라이브러리를 사용하여 SVD(Singular Value Decomposition)와 같은 알고리즘을 통해 RMSE(Root Mean Square Error)를 최소화하고 보다 나은 추천을 할 수 있도록 진행할 것이다.

In [64]:
reader = Reader()

In [65]:
ratings = pd.read_csv('./ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [66]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [67]:
svd = SVD()
cross(svd, data, measures=['RMSE', 'MAE'],cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8936  0.9042  0.8942  0.8973  0.8910  0.8961  0.0045  
MAE (testset)     0.6894  0.6954  0.6848  0.6940  0.6874  0.6902  0.0040  
Fit time          3.70    3.48    3.50    3.53    3.44    3.53    0.09    
Test time         0.16    0.13    0.13    0.12    0.12    0.13    0.01    


{'test_rmse': array([0.89357437, 0.90415137, 0.8942302 , 0.89734283, 0.89102154]),
 'test_mae': array([0.68943658, 0.695418  , 0.68482419, 0.69395153, 0.68737867]),
 'fit_time': (3.6979992389678955,
  3.4780032634735107,
  3.5019984245300293,
  3.5260009765625,
  3.43500018119812),
 'test_time': (0.15700006484985352,
  0.1289970874786377,
  0.12700176239013672,
  0.11799979209899902,
  0.12000036239624023)}

RMSE의 수치들이 충분히 작으므로, 이제 데이터 셋을 train하고 검토해보자.

In [68]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [69]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [70]:
svd.predict(1, 302, 3)#(user Id, Movie ID, Rating)

Prediction(uid=1, iid=302, r_ui=3, est=2.7272820873176515, details={'was_impossible': False})

ID가 302인 영화의 경우 estimated prediction은 3.014로 예상된다. 하지만 이 모델은 영화가 무엇인지(또는 그 안에 무엇이 들어있는지) 상관하지 않는다는 맹점이 존재한다. 이 시스템은 순전히 할당된 영화 ID에 근거하여 작동하며, 다른 사용자들이 어떻게 영화를 평가했는지에 따라 rating을 예측하려고 노력한다.  
하지만 예상하는 바와 같이, 이 모델은 하나의 독립적인 추천 시스템으로 활용하기엔 실용성이 많이 떨어진다. 이를 활용해 마지막으로 하이브리드 추천 시스템을 구상해보자.

## 5. 하이브리드 추천 시스템

이 섹션에서는 콘텐츠 기반 및 협업 필터 기반 추천 시스템에서 구현한 기법을 조합한 단순한 하이브리드 추천 시스템을 구현하려 한다. 구현할 방법은 다음과 같다.

* Input: 사용자 ID 및 영화 제목
* Output: 해당 특정 사용자의 예상 rating을 기준으로 정렬된 유사한 영화들.

In [71]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [72]:
id_map = pd.read_csv('./links.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [73]:
indices_map = id_map.set_index('id')

In [74]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [75]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
1174,Aliens,3282.0,7.7,1986,679,3.271263
580,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.191053
1211,The Terminator,4208.0,7.4,1984,218,3.048368
19725,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.810622
1996,Return from Witch Mountain,38.0,5.6,1978,14822,2.715083
5080,Hawk the Slayer,13.0,4.5,1980,25628,2.701824
21891,Jupiter Ascending,2816.0,5.2,2015,76757,2.655518
30334,Star Trek: Renegades,40.0,4.4,2015,329637,2.651735
12982,Chandu the Magician,6.0,5.3,1932,89573,2.651735
42530,T2 3-D: Battle Across Time,29.0,7.0,1996,65595,2.651735


In [76]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
1174,Aliens,3282.0,7.7,1986,679,3.478773
19725,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.426329
1996,Return from Witch Mountain,38.0,5.6,1978,14822,3.077039
6899,Hercules in New York,63.0,3.7,1969,5227,3.009934
9589,Aliens of the Deep,19.0,6.8,2005,22559,2.965936
37094,The Nostalgist,5.0,7.6,2014,315723,2.965936
24379,Justice League,0.0,0.0,2017,141052,2.965936
21068,Gamera the Brave,11.0,5.9,2006,60160,2.965936
12982,Chandu the Magician,6.0,5.3,1932,89573,2.965936
42530,T2 3-D: Battle Across Time,29.0,7.0,1996,65595,2.965936


이와 같이, 하이브리드 추천 시스템의 경우 영화가 같지만 사용자마다 다른 영화들이 추천되는 것을 확인했다. 따라서 하이브리드 추천 시스템은 보다 개인화되고 특정 사용자에 맞게 조정된다.

### 결론  
이 프로젝트에서는 다양한 방법들을 통해 5개의 다른 추천 시스템을 구상했다. 

1. **Simple 추천 시스템:** 이 시스템은 TMDB vote count와 vote average를 사용하여 일반적으로, 특정 장르에 대해 상위 영화 차트를 추천했다. 가중치 부여 시스템은 정렬이 최종적으로 수행된 rating을 계산하는 데 사용되었다.
2. **Pearson Correlation 추천 시스템:** 이 시스템은 피어슨 상관계수를 이용하여 다른 사용자들이 기존에 평가한 영화들의 평점들을 비교하여 영화를 추천해 주는 시스템이다. 
3. **콘텐츠 기반 추천 시스템:** 여기서는 영화의 (Overview와 Tagline)/(출연진, 제작진, 장르, 키워드)를 활용한 2개의 콘텐츠 기반 엔진과 그것을 보완시킬 수 있는 요소(인기도, 평점)를 추가했다.
4. **협업 필터링 추천 시스템:** Surprise 라이브러리를 사용하여 SVD를 기반으로 한 협업 필터를 구축했다. 획득한 RMSE는 1 미만이었고 엔진은 주어진 사용자 및 영화에 대해 estimated rating을 부여했다.
5. **하이브리드 추천 시스템:** 콘텐츠와 협업 필터링을 통해 특정 사용자에게 영화 제안을 제공하는 엔진을 구축했다.(보다 개인화 시킬 수 있었음)  
  
 

#### 마치며
위의 제시된 모든 엔진을 합쳐 하나의 정교한 추천 시스템을 만들고자 노력했지만, 능력과 시간의 한계로 이는 성공하지 못했다. 하이브리드 추천 시스템으로 이를 실현하고자 하였지만, 막상 구상하고나서 보니 모델의 특성상 사용하려는 사용자 역시 많은 영화들에 대한 평점을 입력을 해야하는데, 이를 단기간에 실천하기는 어려워 보인다(**Cold Start**의 문제점). 오히려 사용자 기반 추천 시스템과 콘텐츠 기반 추천 시스템이 활용도 면에서는 훨씬 낫다고 판단된다. 이번 연구를 진행하며 정말 많은 공부를 했지만, 더 열심히 많은 공부를 해야겠다고 느낀 한 학기였다.